# Lectura del dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report

import joblib

In [ ]:
df_train = pd.read_csv('data/diabetes_prediction_dataset_train-labeled.csv')
df_test = pd.read_csv('data/diabetes_prediction_dataset_test.csv')
df = pd.concat([df_train, df_test])

In [ ]:
plt.rcParams["font.size"] = 20

# Exploración

## Dataset

In [ ]:
df

Vemos que no hay nulos, salvo aquellos que fueron eliminados a propósito.

In [ ]:
df.info()

Separamos en variables numéricas y categóricas.

In [ ]:
cat_cols = ['gender', 'smoking_history']
num_cols = [x for x in df.columns if x not in cat_cols and x not in ['patient', 'diabetes']]
# En las columnas numéricas quitamos la columna "patient" que contiene el id de los pacientes y "diabetes" que es la variable target

## Variables categóricas

Se puede observar que cada variable categórica posee pocas categorías. Además se observa que son variables nominales.

In [ ]:
for col in cat_cols:
    plt.figure(figsize=(10,4))
    sns.histplot(data=df, x=col, stat='density')
    plt.show()

## Variables numéricas

En ningún caso se observan outliers significativos, por lo que se decide no realizar ninguna eliminación ni imputación.

In [ ]:
df[num_cols].describe().T

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(37.5, 20))
a = 0
for col in num_cols[:3]:
    sns.histplot(data=df, x=col, ax=axs[0, a])
    axs[0,a].set_ylabel('')
    a += 1

a = 0
for col in num_cols[3:]:
    sns.histplot(data=df, x=col, ax=axs[1, a])
    axs[0,a].set_ylabel('')
    a += 1

plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(37.5, 20))
a = 0
for col in num_cols[:3]:
    sns.boxplot(data=df, x=col, ax=axs[0, a])
    axs[0,a].set_ylabel('')
    a += 1

a = 0
for col in num_cols[3:]:
    sns.boxplot(data=df, x=col, ax=axs[1, a])
    axs[0,a].set_ylabel('')
    a += 1

plt.show()

## Variable objetivo

Las clases están super desbalanceadas.

In [ ]:
plt.figure(figsize=(10,4))
sns.histplot(data=df, x='diabetes')
plt.show()

# Preprocesamiento

Armamos un pipeline de preprocesamiento, el cual se encarga de:
* Mapeamos las variables categíricas a numéricas utilizando one-hot encoding.
* Normalizamos las variables categóricas.

In [ ]:
ohe = OneHotEncoder(drop='first')
scaler = StandardScaler()
pipeline = ColumnTransformer(
    [("OHE"  , ohe  , cat_cols),
     ("StandardScaler", scaler, num_cols)
    ]
  )
pipeline

In [ ]:
pipeline = joblib.dump(pipeline, 'preproc_pipeline.pkl')